In [4]:
import yfinance as yf
import pandas as pd

In [5]:
start, end = "2009-01-01", "2018-01-01"
Russell = yf.download("^RUA", start=start, end=end, progress= False)


In [6]:
vix = pd.read_excel("VIXCLS.xls")
m2 = pd.read_excel("WM2NS.xls")
unemp_rate = pd.read_excel("U2RATE.xls")
con_sentiment = pd.read_excel("UMCSENT.xls")
t10y2y = pd.read_excel("T10Y2Y.xls")
ppi = pd.read_excel("PPIACO.xls")
fed = pd.read_excel('FEDFUNDS.xls')
cpi = pd.read_excel("CPIAUCSL.xls")

vix = vix.set_index('observation_date')
m2 = m2.set_index('observation_date')
unemp_rate = unemp_rate.set_index('observation_date')
con_sentiment = con_sentiment.set_index('observation_date')
t10y2y = t10y2y.set_index('observation_date')
ppi = ppi.set_index('observation_date')
fed = fed.set_index('observation_date')
cpi = cpi.set_index('observation_date')


vix = vix.resample('D').ffill()
m2 = m2.resample('D').ffill()
unemp_rate = unemp_rate.resample('D').ffill()
con_sentiment = con_sentiment.resample('D').ffill()
t10y2y = t10y2y.resample('D').ffill()
ppi = ppi.resample('D').ffill()
fed = fed.resample('D').ffill()
#cpi = cpi.resample('D').ffill()


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
cpi = cpi.resample('D').ffill()

In [ ]:
cpi

,CPIAUCSL
observation_date,
1947-01-01,21.480
1947-01-02,21.480
1947-01-03,21.480
1947-01-04,21.480
1947-01-05,21.480
...,...
2023-10-28,307.619
2023-10-29,307.619
2023-10-30,307.619


In [ ]:
Russell = pd.merge(Russell,vix,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,m2,how='left',left_index=True, right_index=True)

Russell = pd.merge(Russell,unemp_rate,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,con_sentiment,how='left',left_index=True, right_index=True)


Russell = pd.merge(Russell,t10y2y,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,ppi,how='left',left_index=True, right_index=True)


Russell = pd.merge(Russell,fed,how='left',left_index=True, right_index=True)
Russell = pd.merge(Russell,cpi,how='left',left_index=True, right_index=True)

In [ ]:
Russell = Russell.drop(['Adj Close','Volume'],axis=1)

In [ ]:
Russell.to_csv('data.csv')

In [ ]:
Russell

,Open,High,Low,Close,Adj Close,Volume,VIXCLS,WM2NS,unemployement_rate,consumer_sentiment,T10Y2Y,PPIACO,FEDFUNDS,CPIAUCSL
Date,,,,,,,,,,,,,,
2018-01-02,1585.089966,1595.939941,1585.089966,1595.869995,1595.869995,0,9.77,13962.7,2.0,95.7,0.54,197.900,1.41,248.859
2018-01-03,1595.839966,1605.890015,1595.839966,1605.189941,1605.189941,0,9.15,13962.7,2.0,95.7,0.50,197.900,1.41,248.859
2018-01-04,1605.699951,1613.839966,1605.699951,1611.020020,1611.020020,0,9.22,13962.7,2.0,95.7,0.50,197.900,1.41,248.859
2018-01-05,1612.140015,1621.479980,1612.140015,1621.369995,1621.369995,0,9.22,13962.7,2.0,95.7,0.51,197.900,1.41,248.859
2018-01-08,1621.280029,1624.780029,1617.640015,1624.349976,1624.349976,0,9.52,13969.3,2.0,95.7,0.53,197.900,1.41,248.859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,2769.340088,2803.879883,2769.340088,2803.739990,2803.739990,0,17.68,21741.7,1.9,70.6,0.72,241.338,0.08,280.887
2021-12-28,2803.899902,2813.590088,2795.709961,2798.350098,2798.350098,0,17.54,21741.7,1.9,70.6,0.75,241.338,0.08,280.887
2021-12-29,2798.379883,2807.370117,2791.860107,2801.870117,2801.870117,0,16.95,21741.7,1.9,70.6,0.80,241.338,0.08,280.887
